<a href="https://colab.research.google.com/github/muralikrishnasn/3DNet/blob/master/llamaindex_llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install -q pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip install -q transformers einops accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.7 MB/s eta 0:00:00


In [ ]:
!pip install -q sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.9 MB/s eta 0:00:00


In [ ]:
!pip install -q llama-index-embeddings-huggingface
!pip install -q llama-index-llms-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.3/320.3 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 5.5 MB/s eta 0:00:00


In [ ]:
!pip install -q llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.2 MB/s eta 0:00:00


In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:
documents = SimpleDirectoryReader("/content/data").load_data()

In [ ]:
system_prompt = """
You are a Q&A assistant acting on behalf of a college.
Your goal is to answer questions as accurately
as possible based on the instructions and context provided.
"""

query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.01, "do_sample": True},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
embed_model = HuggingFaceEmbedding(model_name='sentence-transformers/all-mpnet-base-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-13-6394536e40c5>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query('When is the holiday for Ramzan scheduled on?')
print(response)

The holiday for Ramzan is scheduled on April 11, 2024.


In [ ]:
response = query_engine.query('Is the holiday applicable to all?')
print(response)

Yes, the holiday is applicable to all students of the institute. As per the notification, the institute will observe a holiday on April 11, 2024, on account of Ramzan as per MAHE circular dt. November 24, 2023. Therefore, all students are requested to note that April 10, 2024, is a working day, and all classes will be held as per the timetable.


In [ ]:
response = query_engine.query('Why is the college following Saturday\'s timetable on February 21st, Wednesday?')
print(response)

The college is following Saturday's timetable on February 21st, Wednesday, to compensate for the missed classes during the semester due to holidays. As per the notification, the institute had observed holidays on Fridays and Saturdays, which resulted in missed classes for Theory and Laboratory sessions. To make up for these missed classes, the college is scheduling the Saturday timetable on Wednesday, February 21st.


In [ ]:
response = query_engine.query('What is the assessment plan for the sixth semester students?')
print(response)

The assessment plan for the sixth semester students includes the following components:

1. MISAC 1: Surprise Assignment (5 marks) - Conducted during the week as mentioned in Table 1, with a duration of 20 minutes. Students are allowed to refer their handwritten notes, but electronic gadgets, photocopies of notes, or notes written on sheets are not allowed.
2. MISAC 2: Quiz/MCQs (5 marks) - Conducted during the regular class hours, with a duration of 15 minutes. The syllabus for this component covers the topics from January 29 to February 16, 2024.
3. MISAC 3: Mid-term Examination (30 marks) - Conducted during the week as mentioned in Table 1, with a duration of 30 minutes. The syllabus for this component covers the topics from January 03 to March 09, 2024.

In addition, one of the components mentioned in Table 3 (Flexible In -Semester Assessment Component or FISAC


In [ ]:
response = query_engine.query('What are FISACs?')
print(response)

FISACs are Flexible In -Semester Assessment Components. They are one of the types of assessment components in the semester system. FISACs are selected by the faculty and are different from the mandatory In -Semester Assessment Components (MISACs). FISACs are designed to assess the students' understanding of the course content and are conducted in a flexible mode, which means that the students have to complete the assignment within a specified time frame, but the assignment can be submitted anytime within that frame. The details of FISACs are provided in Table 3.


In [ ]:
response = query_engine.query('What is the current protocol for missing any one of the assesments?')
print(response)


If a student misses any one of the mandatory in-semester assessment components (MISAC 1, MISAC 2, or FISAC), the current protocol is as follows:

1. If the student misses any one of the MISAC components for genuine reasons, they will be awarded prorated marks based on the marks scored in the other two components. The prorating will be done as per the examples provided in the context information.
2. If the student misses FISAC, they will have to submit an assignment consisting of three questions in that course.
3. If a student misses any two components of the assessments, they will receive zero marks for the component they missed. However, they will be awarded prorated marks for the other component based on the marks scored in the other two components.

It is important to note that no prorating is permitted if a student misses FISAC and if it is C/D/E/_G/H as per table 3. Additionally, the student has to submit an assignment consisting of five questions in that course.
